### Script to extract data and tidy it for FRAP
Working from data exported by the Nikon software into a text file, we transform it into .csv files, then tidy and output in an excel file

If the Stimulation, Background and References columns are mixed, meaning not in the order S, R , B the script will manage to exchange R and B columns,

but is not able to recognize if S and R are mixed so you will need to change the .csv file with the mixed columns accordingly and rerun the last "tidy" step

In [ ]:
import re
import os
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt

Insert the path to your directory to analyze

In [ ]:
directory_path = r"\\nasac-m2.unige.ch\m-GGotta\GGotta\Vaudano\Data\Microscopy\Live\FRAP\Strain\LP373\MEX5\20231030mNGmex-5_MEX5\Post"

Conversion of the .txt files to csv, be careful that there are no other .txt files than the one exported from Nikon in the folder as it could cause a crash. 

The cell will print the columns as a list when the Stimulation is taking place so you can see if the columns are in order

In [ ]:
for filename in os.listdir(directory_path):
    if filename.endswith(".txt"):
        print(filename)
        with open(os.path.join(directory_path, filename), 'r') as txt_file:
            lines = txt_file.readlines()

            header = ["Name,Time [s],ND.T,ND.Z,ND.M,#1 (EGFP/pH 7),#2 (EGFP/pH 7) (R),#3 (EGFP/pH 7) (B)\n"]

        if "Name" not in lines[0]:  # if header not in file because of Nikon export
            lines = header + lines
        else:
            lines[0] = header[0] # standardize header between files

        with open(os.path.join(directory_path, f"{filename[:-4]}.csv"), 'w') as csv_file:
            lines = [re.sub("\t", ",", line) for line in lines]  # list comprehension to replace tab by comma
            csv_file.writelines(lines)
            
        print("S, R , B ?\n" + str(lines[5].split(",")[-3:]) +"\n" 
              + str(lines[6].split(",")[-3:]) +"\n")

When you open the output file after running the code block below, there will be an empty "sheet1" sheet that you can remove, this is normal

Also the name of the files used to calculate the Reff of the Stim area should start with "reff" so as not to be recognized by this script (because they are also .csv)

In [ ]:
wb = openpyxl.Workbook()
wb.save(filename=os.path.join(directory_path, 'output.xlsx'))

In [ ]:
import scipy.optimize

def Exponential(x, m, k):
    return m * np.exp(-k * x)

def tidy(file_path, output_dir):
    df = pd.read_csv(file_path)

    if df["#2 (EGFP/pH 7) (R)"].values[0] < df["#3 (EGFP/pH 7) (B)"].values[0]:  # switching columns B and R if they are mixed
        df.rename(columns={"#2 (EGFP/pH 7) (R)": "#3 (EGFP/pH 7) (B)", "#3 (EGFP/pH 7) (B)": "#2 (EGFP/pH 7) (R)"}, inplace=True)

    df.rename(columns={"#1 (EGFP/pH 7)": "S", "#2 (EGFP/pH 7) (R)":"R", "#3 (EGFP/pH 7) (B)": "B"}, inplace=True)  # renaming to a S, R and B
    df.drop(columns=['ND.T', 'ND.Z', 'ND.M'], inplace=True)  # removing unnecessary columns

    name = df["Name"][0]  # name to give to the sheet
    # below are the new columns creation
    df["time to zero"] = df["Time [s]"].values - df["Time [s]"].values[5]
    df["S-B"] = df["S"].values - df["B"].values
    df["R-B"] = df["R"].values - df["B"].values
    df["S to zero"] = df["S-B"].values - df["S-B"].values[5]

    S_pre = np.empty(df.shape[0])
    S_pre.fill(0)
    S_pre[0] = np.mean(df["S to zero"].values[0:5])
    df["S preFRAP"] = S_pre
    df["S to one"] = df["S to zero"] / S_pre[0]

    params, cv = scipy.optimize.curve_fit(f=Exponential, xdata=df["time to zero"].values, 
    ydata=df["R-B"].values, p0= [0, 0], bounds=(-np.inf, np.inf))
    m, k = params

    print(name)
    print(f"{m} * np.exp(-{k} * x)")

    try:
        plt.clf()
    except:
        pass

    fig, ax = plt.subplots()
    ax.plot(df["time to zero"].values, df["R-B"].values, '.', label="data")
    ax.plot(df["time to zero"].values, Exponential(df["time to zero"].values, m, k), '--', label="fitted")
    ax.set_title(f"Curve of {name}\n{m} * exp(-{k} * t)")
    ax.set_ylabel("R-B")
    ax.set_xlabel("Time")
    ax.legend()
    fig.savefig(os.path.join(directory_path, "plots", f"{name}.png")) 

    df["S norm"] = df["S to one"] / np.exp(-k * df["time to zero"])

    with pd.ExcelWriter(os.path.join(output_dir, 'output.xlsx'), mode='a') as writer:  
        sheet_name = name.split("_")
        sheet_name = sheet_name[0] + "_" +  sheet_name[-1]
        df.to_excel(writer, sheet_name=f"{sheet_name[:-4]}", 
                columns=['Name', 'Time [s]', 'S', 'R', 'B', 'time to zero', 'S-B', 'R-B',
    'S to zero', 'S preFRAP', 'S to one', 'S norm'], index=False)
        print(f"Wrote {name} to excel with sheet name {sheet_name[:-4]}\n")

    #print(df.head())

# main
try:
    print("Creating plots directory")
    os.makedirs(os.path.join(directory_path, "plots"))
except FileExistsError:
    print("plots directory already created...\n")

for filename in os.listdir(directory_path):
    if filename.endswith(".csv") and not filename.startswith("reff"):
        tidy(os.path.join(directory_path, filename), directory_path)

print("DONE !")